# Challenge Part 2

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import os

# Import the API key.
from config import g_key

In [2]:
# Load the data
file_to_read=os.path.join("weather_data","WeatherPy_database.csv")
city_data_df = pd.read_csv(file_to_read)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Saldanha,ZA,-33.01,17.94,59.00,77,33,5.82,scattered clouds,0.0,0.0
1,1,Butaritari,KI,3.07,172.79,82.99,75,5,18.03,clear sky,0.0,0.0
2,2,Ushuaia,AR,-54.80,-68.30,37.40,64,75,14.99,broken clouds,0.0,0.0
3,3,Machinga,MW,-15.17,35.30,67.24,60,28,2.73,scattered clouds,0.0,0.0
4,4,New Norfolk,AU,-42.78,147.06,37.99,93,89,2.57,overcast clouds,0.0,0.0


In [3]:
# Checking data format before filtering
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [4]:
# Ask the customer to add preferences.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_input = input("Do you want it to be raining? (yes/no) ")
snow_input = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [5]:
# Checking raining and snowing inputs
valid_inputs = ["yes","no"]
if rain_input not in valid_inputs:
    print("Raining input not valid, please try again or the parameter will be ignored!")
if snow_input not in valid_inputs:
    print("Snowing input not valid, please try again or the parameter will be ignored!")

In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
if rain_input == "yes":
    preferred_cities_df=preferred_cities_df.loc[preferred_cities_df["Rain (inches)"]>0]
    if snow_input =="yes":
        preferred_cities_df=preferred_cities_df.loc[preferred_cities_df["Snow (inches)"]>0]
    elif snow_input =="no":
        preferred_cities_df=preferred_cities_df.loc[preferred_cities_df["Snow (inches)"]==0]
elif rain_input == "no":
    preferred_cities_df=preferred_cities_df.loc[preferred_cities_df["Rain (inches)"]==0]
    if snow_input == "yes":
        preferred_cities_df=preferred_cities_df.loc[preferred_cities_df["Snow (inches)"]>0]
    elif snow_input =="no":
        preferred_cities_df=preferred_cities_df.loc[preferred_cities_df["Snow (inches)"]==0]    
        
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
5,5,Albany,US,42.60,-73.97,68.00,90,100,2.19,light rain,0.51,0.0
7,7,Hermanus,ZA,-34.42,19.23,55.99,82,76,3.00,light rain,0.11,0.0
8,8,Busselton,AU,-33.65,115.33,61.00,73,100,23.87,light rain,0.57,0.0
21,21,Nyurba,RU,63.28,118.33,60.15,85,17,0.69,light rain,0.13,0.0
40,40,Dunedin,NZ,-45.87,170.50,50.00,93,100,6.73,light rain,0.32,0.0


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Albany,US,68.00,light rain,42.60,-73.97,
7,Hermanus,ZA,55.99,light rain,-34.42,19.23,
8,Busselton,AU,61.00,light rain,-33.65,115.33,
21,Nyurba,RU,60.15,light rain,63.28,118.33,
40,Dunedin,NZ,50.00,light rain,-45.87,170.50,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through the DataFrame to retrieve hotel info
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        city_error = hotel_df.loc[index,"City"]
        hotel_df.loc[index, "Hotel Name"] = "**No lodging found**"
        print(f"Hotel not found in {city_error}... skipping.")
        pass
hotel_df.head()

Hotel not found in Albany... skipping.
Hotel not found in Chara... skipping.
Hotel not found in Yaan... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Albany,US,68.00,light rain,42.60,-73.97,**No lodging found**
7,Hermanus,ZA,55.99,light rain,-34.42,19.23,Misty Waves Boutique Hotel
8,Busselton,AU,61.00,light rain,-33.65,115.33,Observatory Guest House
21,Nyurba,RU,60.15,light rain,63.28,118.33,Gostinitsa Alrosa
40,Dunedin,NZ,50.00,light rain,-45.87,170.50,Scenic Hotel Southern Cross


In [10]:
# Create the template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Mapping the results using Google Maps API
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file,index_label="City_ID")